In [6]:
# Step 1: Upload the image
from google.colab import files
uploaded = files.upload()

# Step 2: Ensure OpenCV and other dependencies are installed
!pip install opencv-python opencv-python-headless

# Step 3: Import libraries and run the pose detection script
import cv2 as cv
import numpy as np

# Define the body parts and pose pairs
BODY_PARTS = {
    "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
    "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
    "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
    "LEye": 15, "REar": 16, "LEar": 17, "Background": 18
}

POSE_PAIRS = [
    ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
    ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
    ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
    ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
    ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"]
]

# Load the neural network
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")

# Step 4: Update the image path
image_path = list(uploaded.keys())[0]  # Automatically get the uploaded file name
frame = cv.imread(image_path)

if frame is None:
    print(f"Error: Unable to load image '{image_path}'")
    exit()

# Prepare the image for pose detection (similar as before)
frameWidth = frame.shape[1]
frameHeight = frame.shape[0]
net.setInput(cv.dnn.blobFromImage(frame, 1.0, (368, 368), (127.5, 127.5, 127.5), swapRB=True, crop=False))
out = net.forward()
out = out[:, :19, :, :]

points = []
for i in range(len(BODY_PARTS)):
    heatMap = out[0, i, :, :]
    _, conf, _, point = cv.minMaxLoc(heatMap)
    x = (frameWidth * point[0]) / out.shape[3]
    y = (frameHeight * point[1]) / out.shape[2]
    points.append((int(x), int(y)) if conf > 0.2 else None)

# Draw the pose skeleton
for pair in POSE_PAIRS:
    partFrom, partTo = pair
    idFrom, idTo = BODY_PARTS[partFrom], BODY_PARTS[partTo]
    if points[idFrom] and points[idTo]:
        cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
        cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
        cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

# Save and display the result
output_path = "pose_output.png"
cv.imwrite(output_path, frame)
print(f"Pose detection completed. Output saved as '{output_path}'")



Saving image.jpg to image.jpg
Pose detection completed. Output saved as 'pose_output.png'
